In [ ]:
import jpype as jp
import pandas as pd
import string
import re
from nltk.tokenize import TweetTokenizer

In [ ]:
ZEMBEREK_PATH = '...\\bin\\zemberek-full.jar'

# Start the JVM
jp.startJVM("...\\jvm.dll", "-ea", "-Djava.class.path=%s" % (ZEMBEREK_PATH))

In [ ]:
df = pd.read_excel('...xlsx')

In [ ]:
punctuation_removed = [sentence.translate(str.maketrans('', '', string.punctuation)) for sentence in df.text]

In [ ]:
numbers_removed = [re.sub('[0-9]', '', sentences) for sentences in punctuation_removed]

In [ ]:
def turkishdeasciifier(x):
    if "Ã¶" in x:
        x = x.replace("Ã¶", "ö")
    if "Ã¼" in x:
        x= x.replace("Ã¼", "ü")
    if "Ã§" in x:
        x= x.replace("Ã§", "ç")
    if "ÅŸ" in x:
        x= x.replace("ÅŸ", "ş")
    if "Ä±" in x:
        x= x.replace("Ä±", "ı")
    if "ÄŸ" in x:
         x= x.replace("ÄŸ", "ğ")
    if "Ä°" in x:
        x = x.replace('Ä°', 'İ')
    return x

In [ ]:
file = open("stopwordsturkish.txt", "r")
content = file.read()
stop_words_list = content.split("\n")
stop_words_list_deasciified = set([turkishdeasciifier(stop_words) for stop_words in stop_words_list if not stop_words == ''])
stop_words_removed = [' '.join((set(sentences.split(" ")) - stop_words_list_deasciified)) for sentences in numbers_removed]

In [ ]:
def stemmer(token):
    TurkishMorphology = jp.JClass('zemberek.morphology.TurkishMorphology')
    morphology = TurkishMorphology.createWithDefaults()
    results = morphology.analyze(token)
    investigate_result = str(results)[str(results).index('analysisResults=[')+17:-1]
    decided_analysis = investigate_result.split(", ")[0]
    morphology = decided_analysis[decided_analysis.index("] ")+2:]
    stem = morphology[:morphology.index(":")]
    return stem

In [ ]:
tknzr = TweetTokenizer()
stemmed = []
for sent in stop_words_removed:
    stemmed_sentence = ""
    for tokens in tknzr.tokenize(sent):
        try:
            stemmed_sentence = stemmed_sentence + stemmer(tokens) + " "
        except:
            stemmed_sentence = stemmed_sentence + tokens + " "
    stemmed.append(stemmed_sentence)
print(stemmed)